#### 문제 1. pdf 형식의 파일 다운로드 받기
goole 사이트에서 특정 키워드로 검색한 후 pdf 파일로 제공되는 파일을
지정한 폴더로 다운로드 받는 크롤러 만든다.

- 지정한 폴더가 존재하지 않으면 생성하여 파일을 다운로드한다

In [23]:
#1. 필요한 모듈과 라이브러리 불러오기
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import os

In [24]:
#2. 검색 키워드, 건수, 저장경로 입력받기
query_text = input("크롤링할 키워드는 무엇입니까? : ")
cnt = input("크롤링할 건수는 몇 건입니까? : ")
f_dir = input("파일을 저장할 경로만 입력해주세요 (ex : C:\\LHR\\python\\notebook\\practice\\practice4\\ ) : ")

#저장될 경로가 존재하는지 확인
if os.path.isdir(f_dir): 
    print(f"입력할 경로가 존재하여 {f_dir} 폴더에 저장합니다.")
else: #없으면 폴더 생성
    os.makedirs(f_dir)
    print(f'입력한 경로가 존재하지 않아 {f_dir} 을 생성합니다.')

크롤링할 키워드는 무엇입니까? : 데이터분석 pdf
크롤링할 건수는 몇 건입니까? : 10
파일을 저장할 경로만 입력해주세요 (ex : C:\LHR\python\notebook\practice\practice4\ ) : C:\LHR\python\notebook\practice\practice4\
입력할 경로가 존재하여 C:\LHR\python\notebook\practice\practice4\ 폴더에 저장합니다.


In [25]:
#3. 크롬 드라이버를 이용해 웹브라우저를 실행

path = 'C:/LHR/python/datadown/chromedriver.exe'
driver = webdriver.Chrome(path)

#pdf를 다운받을 수 있도록 권한설정 (path 설정이 먼저 되어 있어야함)
driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': f_dir}}
command_result = driver.execute("send_command", params)

driver.get('https://www.google.com/')
time.sleep(2)

In [26]:
#4. 구글 검색 경로 가져와 키워드 입력 및 검색하기
element = driver.find_element_by_css_selector('input.gLFyf.gsfi') #클래스끼리 연결 위해 . 찍기
element.send_keys(query_text)
element.send_keys('\n') #엔터

In [27]:
time.sleep(1)

#5. 페이지 소스 받아오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

pdf_src = soup.select('div.yuRUbf > a') #div 안의 a태그 가져오기

print(len(pdf_src))

cnt = int(cnt)

#수가 내가 지정한 수보다 크면, 내가 지정한 수만큼만 다운받도록
if cnt > len(pdf_src):
    cnt = len(pdf_src)

10


In [10]:
#방법 1
# import urllib.request

# file_no = 0
# pdf_src2 = []
# os.chdir(f_dir) #실행경로는 f_dir로 이동했으므로, 다운은 f_dir에서

# for i in pdf_src: #10번을 반복하며 가져온다
#     try:
#         #바로 해도 되지만, 이해하기 쉽게 분리해서 작성
#         pdf_src1 = i['href'] #a 태그의 속성 하나하나를 담기
#         pdf_src2.append(pdf_src1)
        
#     except Exception as e:
#         print(e)

# print(len(pdf_src2))
# pdf_src2[0]

10


'https://cs.kangwon.ac.kr/~ysmoon/courses/2017_1/grad/06.pdf'

In [28]:
#방법 2
import requests
import shutil #jpg 파일, pdf 파일도 갖고 올 수 있음

file_no = 0
pdf_src2 = []
os.chdir(f_dir) #실행경로는 f_dir로 이동했으므로, 다운은 f_dir에서

for i in pdf_src: #10번을 반복하며 가져온다
    try:
        #바로 해도 되지만, 이해하기 쉽게 분리해서 작성
        pdf_src1 = i['href'] #a 태그의 속성 하나하나를 담기
        pdf_src2.append(pdf_src1)
        
    except Exception as e:
        print(e)

print(len(pdf_src2))
pdf_src2[0]

10


'https://www.kedu.kr/attach/contents/%ED%95%99%EC%8A%B5%EC%9E%90%EB%A3%8C_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%89%BD%EA%B2%8C%20%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0.pdf'

In [29]:
#6. 방법 1 - 다운로드
# for i in range(len(pdf_src2)):
#     if i == cnt: #cnt값과 같으면 반복문을 빠져나온다
#         break
    
#     try:
#         urllib.request.urlretrieve(pdf_src2[i], str(file_no) + '.pdf') #pdf_src2에서 꺼내온 정보를 file_no + .pdf로 저장하겠다
    
#     except Exception as e:
#         print(e)
#         continue #문제가 생겨도 계속 되도록
    
#     file_no += 1
#     time.sleep(0.5)
#     print(f"{file_no} 번째 이미지 저장 중입니다...")

#6. 방법 2 - 다운로드    
for i in range(len(pdf_src2)):
    if i == cnt: #cnt값과 같으면 반복문을 빠져나온다
        break
    
    try:
        res = requests.get(pdf_src2[i], stream = True) #stream 형식이다 (True)
        with open(str(file_no) + '.pdf', 'wb') as f: #쓰기 형식으로 할 거다, f라는 이름으로 할 거다
            res.raw.decode_content = True
            shutil.copyfileobj(res.raw, f) #shutil로 카피
    
    except Exception as e:
        print(e)
        continue #문제가 생겨도 계속 되도록
    
    file_no += 1
    time.sleep(0.5)
    print(f"{file_no} 번째 이미지 저장 중입니다...")

    
#7. 요약정보 출력    
print("=" * 30)
print(f"파일이 저장된 폴더명 : {f_dir}")
print("=" * 30)

driver.close()

1 번째 이미지 저장 중입니다...
2 번째 이미지 저장 중입니다...
3 번째 이미지 저장 중입니다...
4 번째 이미지 저장 중입니다...
5 번째 이미지 저장 중입니다...
6 번째 이미지 저장 중입니다...
7 번째 이미지 저장 중입니다...
8 번째 이미지 저장 중입니다...
9 번째 이미지 저장 중입니다...
10 번째 이미지 저장 중입니다...
파일이 저장된 폴더명 : C:\LHR\python\notebook\practice\practice4\
